In [ ]:
import numpy as np
import matplotlib.mlab as mlab
from gwpy.timeseries import TimeSeries
from scipy import signal
from scipy.fftpack import fft
import datetime as dt
from datetime import datetime

In [ ]:
# define sampling frequency
fs = 4096
#dt = 1/fs

In [ ]:
# GW event time catalog
GW_event = ['Sep 14 2015 09:50:45',
'Oct 12 2015 09:54:43',
'Dec 26 2015 03:38:53',
'Jan 04 2017 10:11:58',
'Jun 08 2017 02:01:16',
'Jul 29 2017 18:56:29',
'Aug 09 2017 08:28:21',
'Aug 14 2017 10:30:43',
'Aug 17 2017 12:41:04',
'Aug 18 2017 02:25:09',
'Aug 23 2017 13:13:58']
# change to datetime data type 
GW_time = [datetime.strptime(i,'%b %d %Y %X') for i in GW_event]

In [ ]:
# define sample time (15s samples 30s before GW events)
delta_0 = dt.timedelta(0,15)
delta_1 = dt.timedelta(0,30)
sample_start = [i-delta_1 for i in GW_time]
sample_end = [i+delta_0 for i in sample_start]
sample_size = len(sample_start)

In [ ]:
# size of data = time * sampling freq
data_size = 15*fs 

In [ ]:
# import data from LIGO H1
data_H1 = np.zeros((sample_size,data_size))
for i in range(sample_size):
    data_H1[i] = TimeSeries.fetch_open_data('H1',sample_start[i], sample_end[i])

In [ ]:
# save H1 data
np.savez("/Users/jkliao117/Desktop/noise time samples H1.npz", data_H1 = data_H1)

In [ ]:
data_H1 = np.load("/Users/jkliao117/Desktop/noise time samples H1.npz")['data_H1']

In [ ]:
# import data from LIGO L1
data_L1 = np.zeros((sample_size,data_size))
for i in range(sample_size):
    data_L1[i] = TimeSeries.fetch_open_data('L1',sample_start[i], sample_end[i])

In [ ]:
# save L1 data
np.savez("/Users/jkliao117/Desktop/noise time samples L1.npz", data_L1=data_L1)

In [ ]:
data_L1 = np.load("/Users/jkliao117/Desktop/noise time samples L1.npz")['data_L1']

In [ ]:
# generate average psd data
def avg_psd(ligo_data):
    (sample_size, data_size) = ligo_data.shape
    # PSD
    NFFT = 4*fs
    psd_window = signal.tukey(NFFT,alpha=0.25)
    NOVL = NFFT/2
    psd = np.zeros((sample_size,int(NOVL+1)))
    for i in range(0,sample_size):
        psd[i], psdfreqs = mlab.psd(ligo_data[i], Fs = fs, NFFT = NFFT, window=psd_window, noverlap=NOVL)
    # average PSD 
    avg_psd = np.sum(psd,axis=0)/psd.shape[0]
    return avg_psd, psdfreqs

In [ ]:
# calculate average psd data for H1 and L1 data
avg_psd_H1, psdfreqs = avg_psd(data_H1)
avg_psd_L1, psdfreqs = avg_psd(data_L1)

In [ ]:
# save avergae psd data for H1 and L1
np.savez("/Users/jkliao117/Desktop/avg psd H1.npz", avg_psd_H1 = avg_psd_H1)
np.savez_compressed("/Users/jkliao117/Desktop/avg psd L1.npz", avg_psd_L1 = avg_psd_L1)
np.savez("/Users/jkliao117/Desktop/psd freqs.npz", psdfreqs = psdfreqs)